# AIOK DE-NAS DEMO

DE-NAS extends AIOK optimized models with Neural Architecture Search Technology. It is a multi-model, hardware-aware, train-free NAS named DE-NAS to construct compact (low computation complexity) model architectures for target platform directly. Firstly, DE-NAS constructs compact neural architecture directly from designed search spaces for multiple domains. Then, a hardware-aware search strategy is used based on given budget to determine the best network. Finally, hardware-aware train-free scoring method is designed to evaluate the candidate network’s performance rather than acquire accuracy of each candidate after its training.

# Content 
* [DE-NAS Architecture and Componements](#DE-NAS-Architecture-and-Componements)
* [Multi-Model Unified Search Space](#Multi-Model-Unified-Search-Space)
    * [CNN-based Search Space](#CNN-based-Search-Space)
    * [Transformer-based Search Space](#Transformer-based-Search-Space)
* [Hardware-Aware Unified Search Framework](#Hardware-Aware-Unified-Search-Framework)
    * [Random Search](#Random-Search)
    * [Evolutionary Search](#Evolutionary-Search)
    * [Bayesian Search (SigOpt-based)](#Bayesian-Search-(SigOpt-based))
* [Train-Free Score](#Train-Free-Score)
* [Try DE-NAS](#Try-DE-NAS)
    * [DE-NAS on CV Domain](#DE-NAS-on-CV-Domain)
    * [DE-NAS on NLP Domain](#DE-NAS-on-NLP-Domain)
    * [DE-NAS on ASR Domain](#DE-NAS-on-ASR-Domain)

# DE-NAS Architecture and Componements

- **Multi-Model Unified Search Space:** DE-NAS supports search spaces including Compute Vision (CV), Natural Language Processing (NLP), and Automatic Speech Recognition (ASR). Meanwhile, the search space utilizes Intel optimized neural operators, and incorporates prior knowledge. Another key innovation is that it is the first of its kind to share a unified transformer-based search space among different models. We leveraged a unified transformer structure for transformer-based search space for CV, NLP and ASR domains.
   
- **Hardware-Aware Unified Search Framework:** DE-NAS provides hardware-aware unifed search framework, and comes with three types of built-in search strategies: Random Search / Evolutionary Search / Bayesian Search, which can cover popular domains of CV/NLP/ASR and enhance DE-NAS with SigOpt-based Bayesian search strategy as an advanced feature.
    
- **Train-Free Evaluation:** DE-NAS provides a novel zero-cost metric called DE-Score, which combines network expressivity, network complexity, network diversity, network saliecny and network latency. Computation of DE-Score only takes a few forward inferences without training, making it extremely fast, lightweight and data-free. 

<img src="./img/denas_framework.jpg" width="80%">
<center>DE-NAS Architecture</center>

# Multi-Model Unified Search Space

## CNN-based Search Space

- Supports CV Domain
- ResNet like skeleton: consists of residual blocks and bottleneck blocks, number of layers, convolutional layer kernel size, number of filters,

<img src="./img/denas_cnn.png" width="50%">
<center>CNN-based Search Space and Supernet</center>

## Transformer-based Search Space 

- Supports CV, NLP, ASR Domain 
- Embedding dimension, number of heads, query/key/value dimension, MLP ratio, network depth.
    
<img src="./img/denas_transformer.png" width="50%">  
<center>Transformer-based Search Space and Supernet</center>

# Hardware-Aware Unified Search Framework

DE-NAS employs a unified search framework for different search strategies. It uses factory design pattern in the DE-NAS search framework, which abstracts the domain-independent search process, and provides three built-in search strategies: Random Search / Evolutionary Search / Bayesian Search that can be easily turned on/off with a switch. Bayesian search algorithm was implemented by leveraging SigOpt API.

<img src="./img/denas_search_framework.jpg" width="50%">
<center>DE-NAS Unified Search Framework</center>

## Random Search
Random Search is the basic search strategy for DE-NAS, which maintains a fixed-size max-heap to store the randomly generated structures. For each randomly generated structure, we compute the hardware-aware train-free score and insert it into the heap. After a given number of iterations, the top of heap is returned as optimal structure.

## Evolutionary Search

<img src="./img/denas_ea.png" width="30%">
<center>Evolutionary Search</center>

- Initialization: Initialize super network architecture with search space, and initialize population pool for neural network architectures.
- Selection: In the selection step, the population pool is sorted according to the calculated hardware-aware train-free score and a new fixed-size population pool is generated.
- Crossover: In the crossover step, two network architecture sequences are selected from the population pool and perform cross over to generate new sequences.
- Mutation: In the mutation step, network architecture sequences of the population pool are randomly flipped.

## Bayesian Search (SigOpt-based)

<img src="./img/sigopt_based_bayesian_search.jpg" width="50%">
<center>SigOpt-based Bayesian Search</center>

# Train-Free Score

The train-free score uses an innovative, zero-cost “proxy” to predict model accuracy instead of full training and validation. It used a novel zero-cost metric combined Gaussian complexity based on network expressivity, NTK score based on network complexity, nuclear norm score based on network diversity, Synflow score based on network saliency, and latency score. The computation of DE-Score only takes a few forward inferences other than iterative training, making it extremely fast, lightweight, and data-free. The overall DE-Score was calculated as following equation:

$DE_{score}=(D_{EXP}+D_{COM}+D_{DIV}+D_{SAL} )∙D_{LAT}$

# Try DE-NAS

## DE-NAS on CV Domain
* [CV](computer_vision/DENAS_COMPUTER_VISION_DEMO.ipynb) - DE-NAS for CV domain

## DE-NAS on NLP Domain
* [NLP](bert/DENAS_BERT_DEMO.ipynb) - DE-NAS for NLP domain

## DE-NAS on ASR Domain
* [ASR](asr/DENAS_ASR_DEMO.ipynb) - DE-NAS for ASR domain